In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [29]:
from datetime import datetime, timedelta
import pandas as pd
import hopsworks

# Define your Hopsworks credentials (either from environment variables or directly)
HOPSWORKS_API_KEY = "11Pnc4YExMWVAIq0.11oLgfRGa7dwpPvQkiu6SA0zwpSAhSQjJ0mSgLdg2wKemRB7roS6culWyM51O3x6"
HOPSWORKS_PROJECT_NAME = "nyc_taxi_proj"

# Connect to Hopsworks
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY, project=HOPSWORKS_PROJECT_NAME)
feature_store = project.get_feature_store()

# Get the current datetime in UTC
current_date = pd.Timestamp.now(tz="Etc/UTC")

# Read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")

# Get the feature view
feature_view = feature_store.get_feature_view(
    name="time_series_hourly_feature_view",  # Make sure this is the correct name
    version=1  # Ensure this matches your feature store version
)

# Fetch batch data
ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)

# Filter data within the range
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]


2025-03-04 19:06:22,877 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 19:06:22,886 INFO: Initializing external client
2025-03-04 19:06:22,886 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 19:06:23,371 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214665
Fetching data from 2025-02-04 00:06:23.792881+00:00 to 2025-03-04 23:06:23.792881+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.08s) 


In [6]:
# from datetime import datetime, timedelta
# import pandas as pd  

# # Get the current datetime64[us, Etc/UTC]  
# current_date = pd.Timestamp.now(tz='Etc/UTC')
# feature_store = get_feature_store()

# # read time-series data from the feature store
# fetch_data_to = current_date - timedelta(hours=1)
# fetch_data_from = current_date - timedelta(days=1*29)
# print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
# feature_view = feature_store.get_feature_view(
#     name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
# )
# ts_data = feature_view.get_batch_data(
#     start_time=(fetch_data_from - timedelta(days=1)),
#     end_time=(fetch_data_to + timedelta(days=1)),
# )
# ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

In [30]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-04 01:00:00+00:00,2,0
1,2025-02-04 02:00:00+00:00,2,0
2,2025-02-04 03:00:00+00:00,2,0
3,2025-02-04 04:00:00+00:00,2,0
4,2025-02-04 05:00:00+00:00,2,0
...,...,...,...
173938,2025-03-04 17:00:00+00:00,263,117
173939,2025-03-04 18:00:00+00:00,263,132
173940,2025-03-04 19:00:00+00:00,263,99
173941,2025-03-04 20:00:00+00:00,263,86


In [31]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 173943 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         173943 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  173943 non-null  int32                  
 2   rides               173943 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [32]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [33]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 173943 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         173943 non-null  datetime64[us]
 1   pickup_location_id  173943 non-null  int32         
 2   rides               173943 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [34]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [35]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [36]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,2,0,0,0,0,3,0,0,3,5,...,3,0,1,0,1,0,0,16,255,2025-02-11 02:00:00
1,0,0,1,0,0,0,6,2,0,0,...,1,0,0,1,0,1,1,1,112,2025-02-28 18:00:00
2,2,1,0,0,1,1,0,1,0,1,...,0,0,1,0,0,2,0,0,218,2025-02-28 19:00:00
3,369,273,249,21,251,348,17,373,286,251,...,222,32,324,393,166,35,89,335,237,2025-02-10 22:00:00
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,57,2025-02-11 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,1,0,0,108,2025-02-10 03:00:00
247,0,0,0,2,0,0,4,0,1,0,...,3,2,0,0,0,2,0,0,71,2025-02-28 14:00:00
248,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8,2025-02-27 23:00:00
249,0,1,1,1,1,1,2,0,4,0,...,2,4,1,0,0,0,0,0,168,2025-02-28 15:00:00


In [20]:
# import pandas as pd

# # Convert timestamps safely
# rides["pickup_datetime"] = pd.to_datetime(rides["pickup_datetime"], errors="coerce")

# # Drop NaT timestamps to remove invalid dates
# rides = rides.dropna(subset=["pickup_datetime"]).copy()

# # Step 1: Ensure timestamps are first localized to UTC
# if rides["pickup_datetime"].dt.tz is None:
#     rides["pickup_datetime"] = rides["pickup_datetime"].dt.tz_localize("UTC")

# rides["pickup_datetime"] = (
#     rides["pickup_datetime"]
#     .dt.tz_localize(None)  # Remove any existing timezone
#     .dt.tz_localize("America/New_York", ambiguous="NaT")  # Mark ambiguous timestamps as NaT
# )

# # Fill missing timestamps by forward or backward filling (choose based on data behavior)
# rides["pickup_datetime"] = rides["pickup_datetime"].fillna(method="ffill")


# # Step 3: Drop any remaining NaT values (previously ambiguous timestamps)
# rides = rides.dropna(subset=["pickup_datetime"])

# # Step 4: Safely floor timestamps to the nearest hour
# rides["pickup_hour"] = rides["pickup_datetime"].dt.floor("h")

# # Debug final data before transformation
# print("Final dataset size before transformation:", rides.shape)
# print(rides[["pickup_datetime", "pickup_hour"]].head())

# # ✅ Now pass only clean data to transformation function
# ts_data = transform_raw_data_into_ts_data(rides)

# print("Transformation complete!")
# print(ts_data.head())


In [37]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-04 19:06:55,473 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 19:06:55,480 INFO: Initializing external client
2025-03-04 19:06:55,481 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 19:06:56,081 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214665
Fetching data from 2025-02-04 00:06:55.473602+00:00 to 2025-03-04 23:06:55.473602+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.15s) 


In [38]:
current_date

Timestamp('2025-03-05 00:06:55.473602+0000', tz='Etc/UTC')

In [39]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-04 01:00:00
1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,3,2025-03-04 01:00:00
2,6,2,0,1,1,2,1,11,7,3,...,1,1,0,0,1,1,1,4,4,2025-03-04 01:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-04 01:00:00
4,0,1,0,0,1,2,2,1,2,3,...,4,1,5,2,1,1,0,0,7,2025-03-04 01:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,259,2025-03-04 01:00:00
247,0,0,0,0,0,1,4,2,0,1,...,6,1,1,0,0,0,1,0,260,2025-03-04 01:00:00
248,3,0,0,2,2,0,4,5,16,21,...,43,45,24,22,21,8,10,1,261,2025-03-04 01:00:00
249,3,1,1,3,19,63,126,177,156,102,...,82,103,65,37,16,14,9,2,262,2025-03-04 01:00:00


In [40]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-04 19:07:14,569 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 19:07:14,574 INFO: Initializing external client
2025-03-04 19:07:14,574 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 19:07:15,145 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214665


In [41]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [42]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,2.0
3,6,0.0
4,7,1.0
...,...,...
246,259,0.0
247,260,0.0
248,261,1.0
249,262,1.0


In [43]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([132, 138, 249,  48, 230, 186, 114,  79, 142,  68], dtype=int32)